In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
if '..' not in sys.path:
    sys.path.append('..')
from run_AC_analysis import compute_fourier_coeffs
from pfcommon import correct_traces

In [ ]:
data_dir = '..'
project_name = 'V2020_Rete_Sardegna_2021_06_03cr_AC' # 'IEEE39_AC' # 'AC_WSCC'
# F_start = 1e-3, 1e-1
# F_stop = 1e1, 1e2
# steps_per_decade = 10, 30
F_start = 1e-3,
F_stop = 1e1,
steps_per_decade = 10,
suffix = ''
data_files = ['{}_{:.0f}_{:.0f}_{}{}.npz'.format(project_name, np.log10(F0), np.log10(F1), steps, suffix) \
              for F0,F1,steps in zip(F_start, F_stop, steps_per_decade)]
n_files = len(data_files)
blob = [np.load(os.path.join(data_dir, f), allow_pickle=True) for f in data_files]
F = [b['F'] for b in blob]
time = [b['time'] for b in blob]
gamma_c,gamma_s = [], []
config = blob[0]['config'].item()
for b in blob:
    speed = np.array([d['gen']['s:xspeed'] for d in b['data']], dtype=object)
    fi = np.array([d['gen']['c:fi'] for d in b['data']], dtype=object)
    Vr = np.array([d['bus']['m:ur'] for d in b['data']], dtype=object)
    Vi = np.array([d['bus']['m:ui'] for d in b['data']], dtype=object)
    fe = np.array([d['bus']['m:fe'] for d in b['data']], dtype=object)
    γc,γs = compute_fourier_coeffs(b['F'], b['time'], speed, mu=config['n_periods'])
    gamma_c.append(γc)
    gamma_s.append(γs)

In [ ]:
fig,ax = plt.subplots(1, 1, sharex=True, figsize=(7,4))
n_generators = gamma_c[0].shape[1]
cmap = plt.get_cmap('viridis', n_generators)
for i in range(n_files):
    for j in range(n_generators):
        dB = 10 * np.log10(np.sqrt(gamma_c[i][:,j]**2 + gamma_s[i][:,j]**2) / F[i])
#         dB = 10 * np.log10(np.sqrt(gamma_c[i][:,j]**2 + gamma_s[i][:,j]**2))
        ax.plot(F[i], dB, '-', color=cmap(j), lw=1, label='G{}'.format(j+1) if i == 0 else None)
ax.set_xscale('log')
ax.set_xlabel('Frequency [Hz]')
ax.set_ylabel('dB10')
sns.despine()
plt.savefig(os.path.splitext(data_files[0])[0] + suffix + '.pdf')

In [ ]:
i = 1
ref_gen = 0
δr = fi[i][:, ref_gen]
Vd,Vq = correct_traces(Vr[i], Vi[i], δr)

In [ ]:
j = 5
fig,ax = plt.subplots(3, 2, sharex=True, figsize=(10,4))
ax[0,0].plot(time[0][i], Vd[:,j], 'k', lw=1)
ax[0,1].plot(time[0][i], Vr[i][:,j], 'r', lw=1)
ax[1,0].plot(time[0][i], Vq[:,j], 'k', lw=1)
ax[1,1].plot(time[0][i], Vi[i][:,j], 'r', lw=1)
ax[2,0].plot(time[0][i], np.sqrt(Vd[:,j]**2 + Vq[:,j]**2), 'k', lw=1)
ax[2,1].plot(time[0][i], np.sqrt(Vr[i][:,j]**2 + Vi[i][:,j]**2), 'r', lw=1)
sns.despine()

In [ ]:
fig,ax = plt.subplots(2, 1, figsize=(6,6), sharex=True)
i,j = 0,0
ax[0].plot(time[0][i], fe[i][:,j], 'k', lw=1)
ax[0].plot(time[0][i], speed[i][:,j], 'r', lw=1)
# ax[1].plot(time[0][i], Vr[i][:,j], 'tab:red', lw=1)
# ax[1].plot(time[0][i], Vi[i][:,j], 'tab:green', lw=1)
ax[1].plot(time[0][i], np.sqrt(Vr[i][:,j]**2 + Vi[i][:,j]**2), 'k', lw=1)